In [1]:
flightData2015 = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv("/tools/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv")

In [2]:
flightData2015.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [3]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [4]:
flightData2015.take(3)


[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [5]:
flightData2015.sort("count").explain()

== Physical Plan ==
*(2) Sort [count#12 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#12 ASC NULLS FIRST, 200)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#10,ORIGIN_COUNTRY_NAME#11,count#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tools/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


By default, when we perform a shuffle, Spark
outputs 200 shuffle partitions. Let’s set this value to 5 to reduce the number of the output
partitions from the shuffle

In [6]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

flightData2015.sort("count").explain()


== Physical Plan ==
*(2) Sort [count#12 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#12 ASC NULLS FIRST, 5)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#10,ORIGIN_COUNTRY_NAME#11,count#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tools/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


In [7]:
# this took 45 ms
spark.conf.set("spark.sql.shuffle.partitions", "200")
flightData2015.sort("count", ascending=False).show()

+------------------+-------------------+------+
| DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+------------------+-------------------+------+
|     United States|      United States|370002|
|     United States|             Canada|  8483|
|            Canada|      United States|  8399|
|     United States|             Mexico|  7187|
|            Mexico|      United States|  7140|
|    United Kingdom|      United States|  2025|
|     United States|     United Kingdom|  1970|
|             Japan|      United States|  1548|
|     United States|              Japan|  1496|
|           Germany|      United States|  1468|
|     United States| Dominican Republic|  1420|
|Dominican Republic|      United States|  1353|
|     United States|            Germany|  1336|
|       South Korea|      United States|  1048|
|     United States|        The Bahamas|   986|
|       The Bahamas|      United States|   955|
|     United States|             France|   952|
|            France|      United States|

In [8]:
# this took 32 ms
spark.conf.set("spark.sql.shuffle.partitions", "5")
flightData2015.sort("count", ascending=False).show()

+------------------+-------------------+------+
| DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+------------------+-------------------+------+
|     United States|      United States|370002|
|     United States|             Canada|  8483|
|            Canada|      United States|  8399|
|     United States|             Mexico|  7187|
|            Mexico|      United States|  7140|
|    United Kingdom|      United States|  2025|
|     United States|     United Kingdom|  1970|
|             Japan|      United States|  1548|
|     United States|              Japan|  1496|
|           Germany|      United States|  1468|
|     United States| Dominican Republic|  1420|
|Dominican Republic|      United States|  1353|
|     United States|            Germany|  1336|
|       South Korea|      United States|  1048|
|     United States|        The Bahamas|   986|
|       The Bahamas|      United States|   955|
|     United States|             France|   952|
|            France|      United States|

Both SQL and DataFrame syntax compile to the same execution plan. So, there's no perf penalty in writing sql.

In [9]:
flightData2015.createOrReplaceTempView("flight_data_2015")

sqlWay = spark.sql("SELECT DEST_COUNTRY_NAME, count(1)\
                    FROM flight_data_2015\
                    GROUP BY DEST_COUNTRY_NAME")

dfWay = flightData2015\
            .groupBy("DEST_COUNTRY_NAME")\
            .count()

sqlWay.explain()
print("-------------------------------------------")
dfWay.explain()




== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#10] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tools/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
-------------------------------------------
== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#10] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tools/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], R

-------------------------------------------------------------------------------

In [16]:
from pyspark.sql.functions import max

flightData2015.select(max("count")).collect()[0][0]

370002

In [19]:
maxSql = spark.sql("""
    SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
    FROM flight_data_2015
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 5
    """)
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [39]:
# same as above but with DataFrame syntax
from pyspark.sql.functions import desc

flightData2015\
    .groupBy("DEST_COUNTRY_NAME")\
    .sum("count")\
    .withColumnRenamed("sum(count)", "destination_total")\
    .sort(desc("destination_total"))\
    .limit(5)\
    .show()


# OR
flightData2015\
    .groupBy("DEST_COUNTRY_NAME")\
    .agg({"count": "sum"})\
    .withColumnRenamed("sum(count)", "destination_total")\
    .orderBy("destination_total", ascending=False)\
    .limit(5)\
    .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [41]:
# let's check the execution plan
from pyspark.sql.functions import desc

flightData2015\
    .groupBy("DEST_COUNTRY_NAME")\
    .sum("count")\
    .withColumnRenamed("sum(count)", "destination_total")\
    .sort(desc("destination_total"))\
    .limit(5)\
    .explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#466L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#10,destination_total#466L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[sum(cast(count#12 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 5)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_sum(cast(count#12 as bigint))])
         +- *(1) FileScan csv [DEST_COUNTRY_NAME#10,count#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tools/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


This addition operation happens because Spark will convert an expression written in an input
language to Spark’s internal Catalyst representation of that same type information. It then will
operate on that internal representation.

In [1]:
a = 10
b = 20
df = spark.range(500).toDF("number")
xz = df.select(df.number, df.number + a, df["number"] + b)
xz.show()

+------+-------------+-------------+
|number|(number + 10)|(number + 20)|
+------+-------------+-------------+
|     0|           10|           20|
|     1|           11|           21|
|     2|           12|           22|
|     3|           13|           23|
|     4|           14|           24|
|     5|           15|           25|
|     6|           16|           26|
|     7|           17|           27|
|     8|           18|           28|
|     9|           19|           29|
|    10|           20|           30|
|    11|           21|           31|
|    12|           22|           32|
|    13|           23|           33|
|    14|           24|           34|
|    15|           25|           35|
|    16|           26|           36|
|    17|           27|           37|
|    18|           28|           38|
|    19|           29|           39|
+------+-------------+-------------+
only showing top 20 rows



In [4]:
    rng = spark.range(5)
    rng.collect()

[Row(id=0), Row(id=1), Row(id=2), Row(id=3), Row(id=4)]

In [5]:
rng = spark.range(5).explain()

== Physical Plan ==
*(1) Range (0, 5, step=1, splits=2)
